In [ ]:
import polars as pl
from datetime import datetime
import get_and_store as gas
import polars_indicators as pi
from polars_indicators import IndicatorResult
DATE_COLUNN = "Date"

In [ ]:
symbols = ['A','AA']

In [ ]:
df = gas.get_symbols_df(symbols)
df = df.filter(pl.col('Date') >= datetime(2021, 1, 1))
df = df.collect()

In [ ]:
df

In [ ]:
sma25 = pi.simple_moving_average(df, days=25)
sma50 = pi.simple_moving_average(sma25.df, days=50)
df = sma50.df
df

In [ ]:
cross_up = pi.crossover_up(df, sma25.column, sma50.column)
cross_up.df

In [ ]:
#convert to open values
cross_up.df = cross_up.df.with_columns(pl.when(pl.col(cross_up.column)).then(pl.col("Open")).alias(cross_up.column))

In [ ]:
trail = pi.trailing_stop(cross_up.df, bars=2)
t = trail.df.filter(pl.col("Date").is_between(datetime(2022, 6, 1), datetime(2022, 6, 12)))
t

In [ ]:
index = "row_nr"
bars = 2
df = t.select(trail.column).with_row_count(index)
df.drop_nulls(trail.column).with_columns((pl.col(index) - pl.col(index).shift(1)).alias("test"))

In [ ]:
#left off here
col1 = 'test'
df = df.drop_nulls(trail.column).with_columns(pl.when(pl.col(index) - pl.col(index).shift(1) < bars).then(pl.col(index).shift(1)).otherwise(pl.col(index)).alias(col1))
df

In [ ]:
#left off here. Maybe something like this is my answer?
col2 = "test2"
df = df.drop_nulls(trail.column).with_columns(pl.when(pl.col(index) - pl.col(index).shift(1) < bars).then(pl.col(col1).shift(1)).otherwise(pl.col(index)).alias(col2))
df

In [ ]:
def limit_entries(df: pl.DataFrame | pl.LazyFrame, bars: int, entries: str) -> IndicatorResult:
    """Forces a minimum number of bars between entries"""
    column_name = f"{bars}_minimum_bars_between"
    if column_name in df.columns:
        return IndicatorResult(df, column_name)
    for i in range(bars, 0, -1):
        df = df.with_columns(pl.col(entries).shift(i))
    return IndicatorResult(df, column_name)
limit_entries(t, 1, trail.column).df

In [ ]:
ret = pi.create_trade_ids(trail.df, cross_up.column, trail.column)

In [ ]:
pi.summarize_trades(ret.df, ret.column, cross_up.column, trail.column)